# Customer Model on GCP 

## Play with BQML

To re-create the BQML recommender in Notebook

Ref: https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-matrix-factorization

From BQML Guide:
* https://cloud.google.com/bigquery-ml/docs/bigqueryml-mf-explicit-tutorial
* https://cloud.google.com/bigquery-ml/docs/bigqueryml-mf-implicit-tutorial


In [ ]:
from google.cloud import bigquery

client = bigquery.Client(location="US")

### Create dataset for data and model

In [ ]:
dataset = client.create_dataset("bqml_recommender")

### Load GoogleAnalytics 360 data into BigQuery 

In [ ]:
%%bigquery

CREATE OR REPLACE TABLE bqml_recommender.analytics_session_data AS
WITH
 visitor_page_content AS (
 SELECT
   fullVisitorID,
   (
   SELECT
     MAX(
     IF
       (index=10,
         value,
         NULL))
   FROM
     UNNEST(hits.customDimensions)) AS latestContentId,
   (LEAD(hits.time, 1)
     OVER (PARTITION BY fullVisitorId ORDER BY hits.time ASC) - hits.time)
             AS session_duration
 FROM
   `cloud-training-demos.GA360_test.ga_sessions_sample`,
   UNNEST(hits) AS hits
 WHERE
   # only include hits on pages
   hits.type = "PAGE"
 GROUP BY
   fullVisitorId,
   latestContentId,
   hits.time )
 # aggregate web stats
SELECT
 fullVisitorID AS visitorId,
 latestContentId AS contentId,
 SUM(session_duration) AS session_duration
FROM
 visitor_page_content
WHERE
 latestContentId IS NOT NULL
GROUP BY
 fullVisitorID,
 latestContentId
HAVING
 session_duration > 0
ORDER BY
 latestContentId


Query directly using `%%bigquery` magic return a Pandas DataFrame to workspace.

Or using the `df = client.query(sql).to_dataframe()` to return the DataFrame

In [ ]:
%%bigquery

SELECT *
FROM bqml_recommender.analytics_session_data
LIMIT 10;

In [ ]:
_

In [ ]:
sql = """
SELECT *
FROM bqml_recommender.analytics_session_data
LIMIT 10;
"""

df = client.query(sql).to_dataframe()
df.head()

### Create and Train Model

Ref: https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-matrix-factorization


In [ ]:
%%bigquery

CREATE OR REPLACE MODEL bqml_recommender.retail_recommender
OPTIONS
  (model_type='matrix_factorization',
   feedback_type='implicit',
   user_col='visitorId',
   item_col='contentId',
   rating_col='rating',
   l2_reg=30,
   num_factors=15) AS
SELECT
  visitorId,
  contentId,
  0.3 * (1 + (session_duration - 57937) / 57937) AS rating
FROM bqml_recommender.analytics_session_data
WHERE 0.3 * (1 + (session_duration - 57937) / 57937) < 1


### Evaluation

Ref: [Ranking metrics](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-evaluate) 

In [ ]:
%%bigquery

SELECT
  *
FROM
  ML.EVALUATE(MODEL `bqml_recommender.retail_recommender`)

### Make Predictions

In [ ]:
%%bigquery

SELECT * FROM 
    ML.RECOMMEND(MODEL bqml_recommender.retail_recommender,
        (
        SELECT visitorId
        FROM bqml_recommender.analytics_session_data
        LIMIT 5
        )
    )

### Confidence Rating of user-item pair for all

This confidence value approximately lies between 0 and 1 where the higher confidence indicates that the user prefers item more than an item with a lower confidence value.

The output is saved in a table for others to query.

In [ ]:
%%bigquery

CREATE OR REPLACE TABLE bqml_recommender.recommend_content
OPTIONS() AS
    SELECT *
    FROM ML.RECOMMEND(MODEL bqml_recommender.retail_recommender)